In [2]:
import xml.etree.ElementTree as ET
import mysql.connector
import logging
import os
from dotenv import load_dotenv
load_dotenv("../config.env",verbose=True)

True

In [3]:
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user=os.environ["sql_user"],
        password=os.environ["sql_password"],
        database="mydb"
    )

ns = {'ns': 'http://www.battlescribe.net/schema/catalogueSchema'}

def insert_faction(root, cursor):
    _name = root.get('name')
    _description = root.get('description')
    if _description is None:
        _description = ""
    _id = root.get('id')

    sql = "INSERT INTO Faction (name, description, id) VALUES (%s, %s,%s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_faction", exc_info=e)
        print(e)

    return _id

def insert_unit2(cursor, root, faction_id):
    sharedSelectionEntries = root.find('ns:sharedSelectionEntries', ns)

    if(sharedSelectionEntries == None):
        return

    for selectionEntry in sharedSelectionEntries:
        if "[Legends]" not in selectionEntry.get("name"):
            if selectionEntry.get("type") == "model" or selectionEntry.get("type") == "unit":

                _name = selectionEntry.get("name")
                _description = selectionEntry.get("description")
                if _description == None:
                    _description = ""
                _id = selectionEntry.get("id")

                sql = "INSERT INTO Unit (name, description, id, faction_id) VALUES (%s, %s,%s,%s)"
                try:
                    print("inserting unit", _name, _description, _id, faction_id)
                    cursor.execute(sql, (_name, _description, _id, faction_id))
                    parse_selectionEntry(cursor, selectionEntry, _id)
                    insert_keywords(cursor, selectionEntry, _id)
                    insert_costs(cursor, selectionEntry, _id)
                except Exception as e:
                    logging.error("Error at %s", "insert_unit", exc_info=e)
                    print(e)

def insert_keywords(cursor, selectionEntry, unit_id):
    keywords = selectionEntry.findall(".//ns:categoryLink", ns)
    for keyword in keywords:
        _name = keyword.get("name")
        _id = keyword.get("targetId")
        sql = "INSERT IGNORE INTO ModelKeyWord (text, id) VALUES (%s, %s)"
        sql_map = "INSERT IGNORE INTO unit_has_ModelKeyword (Unit_id, ModelKeyword_id) VALUES (%s, %s)"
        try:
            cursor.execute(sql, (_name, _id))
            cursor.execute(sql_map, (unit_id, _id))
        except Exception as e:
            logging.error("Error at %s", "insert_keywords", exc_info=e)
            print(e)

def parse_selectionEntry(cursor, selectionEntry, unit_id):
    profiles = selectionEntry.findall(".//ns:profile", ns)
    weapons = []
    for profile in profiles:
        if profile.get("typeName") == "Unit" or profile.get("typeName") == "Model":
            insert_model(cursor, profile, unit_id)
        if profile.get("typeName") == "Abilities":
            insert_abilities(cursor, profile, unit_id)
        if profile.get("typeName") == "Ranged Weapons" or profile.get("typeName") == "Melee Weapons":
            weapons.append(profile)

    for profile in weapons:
        insert_weapons(cursor, profile, unit_id)

def insert_weapons(cursor, profile, unit_id):
    _name = profile.get("name")
    _id = profile.get("id")
    c = profile.find(".//ns:characteristics", ns)

    _range = None
    _strength = None
    _ap = None
    _damage = None
    _attack = None
    _hit = None

    keywords = None

    for characteristic in c:
        if characteristic.get("name") == "Range":
            _range = characteristic.text
        if characteristic.get("name") == "S":
            _strength = characteristic.text
        if characteristic.get("name") == "AP":
            _ap = characteristic.text
        if characteristic.get("name") == "D":
            _damage = characteristic.text
        if characteristic.get("name") == "A":
            _attack = characteristic.text
        if characteristic.get("name") == "WS":
            _hit = characteristic.text
        if characteristic.get("name") == "BS":
            _hit = characteristic.text
        if characteristic.get("name") == "Keywords":
           keywords = characteristic

    sql = "INSERT INTO weapon (name, weapon_range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES (%s, %s,%s,%s,%s,%s,%s,%s,%s)"

    try:
        print("inserting weapon", _name, _hit, _strength, _ap, _damage, _id, _attack, unit_id)
        cursor.execute(sql, (_name, _range, _hit, _strength, _ap, _damage, _id, _attack, unit_id))
    except Exception as e:
        logging.error("Error at %s", "insert_weapons", exc_info=e)
        print("error!!!!")
        print(e)

    if(keywords != None):
        insert_weapon_keywords(cursor, keywords, _id)

def insert_weapon_keywords(cursor, characteristic, weapon_id):
    _keywords = characteristic.text
    # Check if the weapon has keywords
    if _keywords is None:
        return

    if len(_keywords) < 3:
        return
    keywords = characteristic.text.split(", ")

    for keyword in keywords:
        if keyword.replace("/n", "").replace(" ", "") == "":
            continue

        sql = "select * from weaponkeyword where name = %s"
        cursor.execute(sql, (keyword,))
        result = cursor.fetchone()
        if result is None or len(result) == 0:
            sql = "INSERT INTO weaponkeyword (name) VALUES (%s)"
            print("inserting keyword", keyword)
            cursor.execute(sql, (keyword,))
            cursor.execute("SELECT id FROM weaponkeyword WHERE name = %s", (keyword,))
            result = cursor.fetchone()

        sql = "INSERT INTO WeaponHasKeyword (Weapon_id, WeaponKeyword_id) VALUES (%s, %s)"
        print("inserting keyword relationship", weapon_id, result[0])
        cursor.execute(sql, (weapon_id, result[0]))

def insert_abilities(cursor, profile, unit_id):
    _name = profile.get("name")
    _id = profile.get("id")
    _description = profile.findall(".//ns:characteristic", ns)[0].text

    if(_description is None):
        _description = ""

    sql = "INSERT INTO modelability (name, description, id) VALUES (%s, %s,%s)"
    sql_map = "INSERT INTO unit_has_modelability (unit_id, ModelAbility_id) VALUES (%s, %s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
        cursor.execute(sql_map, (unit_id, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_abilities", exc_info=e)
        print(e)

def insert_model(cursor, model, unit_id):
    _name = model.get("name")
    _id = model.get("id")

    move = None
    leadership = None
    save = None
    toughness = None
    wounds = None
    oc = None
    keywords = None
    for characteristics in model.findall(".//ns:characteristic", ns):

        if characteristics.get("name") == "M":
            move = characteristics.text
        elif characteristics.get("name") == "LD":
            leadership = characteristics.text
        elif characteristics.get("name") == "SV":
            save = characteristics.text
        elif characteristics.get("name") == "T":
            toughness = characteristics.text
        elif characteristics.get("name") == "W":
            wounds = characteristics.text
        elif characteristics.get("name") == "OC":
            oc = characteristics.text
        elif characteristics.get("name") == "Keywords":
            keywords = characteristics

        sql_models = "INSERT INTO Model (id, name, move, toughness,wounds, leadership, save, unit_id, oc) VALUES (%s, %s,%s,%s, %s, %s,%s,%s,%s)"

    try:
        print("inserting model", _id, _name, move, toughness, wounds, leadership, save, unit_id, oc)
        cursor.execute(sql_models, (_id, _name, move, toughness, wounds, leadership, save, unit_id, oc))

    except Exception as e:
        logging.error("Error at %s", "insert_model", exc_info=e)
        print(e)

    if(keywords != None):
        insert_keywords(cursor, keywords, unit_id)

def insert_ally(cursor, root):
    r = root.find(".//ns:catalogueLinks", ns)
    if(r is None):
        return
    _id = root.get("id")
    for re in r:
        sql = "select * from faction where name = %s"
        cursor.execute(sql, (re.get("name"),))
        result = cursor.fetchone()
        if(result is not None):
            sql = "insert into CanTakeAsAllied (faction_id, allied_id) values (%s, %s)"
            print("inserting ally relationship", _id, result[0])
            cursor.execute(sql, (_id, result[0]))

def insert_costs(cursor, selectionEntry, unit_id):
    costs = selectionEntry.find(".//ns:costs", ns)
    sql = "insert into UnitCost (unit_id, cost, model_count) values (%s, %s, %s)"
  
    if(costs is not None):
        for c in costs:
            try:
                print("inserting unit cost", unit_id, c.get("value"), "default")
                cursor.execute(sql, (unit_id, c.get("value"), "default"))
            except Exception as e:
                logging.error("Error at %s", "insert_cost_default", exc_info=e)
                print(e)

    modifiers = selectionEntry.find(".//ns:modifiers", ns)

    if modifiers is None:
        return

    for m in modifiers:
        #Modifiers about cost have this id
        if m.get("field") != "51b2-306e-1021-d207":
            continue
        
        values = m.findall(".//ns:condition", ns)    

        for v in values:
            try:
                print("inserting unit cost", unit_id,m.get("value"), v.get("type")+ " " +v.get("value"))
                cursor.execute(sql, (unit_id, m.get("value"),  v.get("type")+ " " +v.get("value")))
            except Exception as e:
                logging.error("Error at %s", "insert_cost_modifier", exc_info=e)
                print(e)



def parse_cat_file(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    return root

def ingest_data(root, cursor):
    faction_id = insert_faction(root, cursor)
    insert_unit2(cursor, root, faction_id)
    insert_ally(cursor, root)
    db_connection.commit()

def clear_all_tables(cursor):
    # Get all table names

    # Iterate over all tables and delete all records
    try:
        cursor.execute(f"DELETE FROM `CanTakeAsAllied`")
        cursor.execute(f"DELETE FROM `WeaponHasKeyword`")
        cursor.execute(f"DELETE FROM `WeaponKeyword`")
        cursor.execute(f"DELETE FROM `Unit_has_ModelKeyword`")
        cursor.execute(f"DELETE FROM `Weapon`")
        cursor.execute(f"DELETE FROM `Model`")
        cursor.execute(f"DELETE FROM `ModelKeyword`")
        cursor.execute(f"DELETE FROM `Unit_has_ModelAbility`")
        cursor.execute(f"DELETE FROM `ModelAbility`")
        cursor.execute(f"DELETE FROM `UnitCost`")
        cursor.execute(f"DELETE FROM `Unit`")
        cursor.execute(f"DELETE FROM `Faction`")

        db_connection.commit()

    except mysql.connector.Error as err:
        print(f"Something went wrong: {err}")


In [4]:
db_connection = connect_to_db()
cursor = db_connection.cursor(buffered=True)

In [5]:
names = os.listdir("./wh40k-10e")
clear_all_tables(cursor)

for n in names:
    if(".cat" not in n):
        continue

    print("ingesting for file: " + n)

    root = parse_cat_file("./wh40k-10e/" + n)
    ingest_data(root, cursor)

for n in names:
    if(".cat" not in n):
        continue
    root = parse_cat_file("./wh40k-10e/" + n)
    insert_ally(cursor, root)
    
db_connection.close()

ingesting for file: Aeldari - Aeldari Library.cat
inserting unit Asurmen  828d-840a-9a67-9074 dfcf-1214-b57-2205
inserting model dd63-cf49-2bda-19d2 Asurmen 7" 3 5 6+ 2+ 828d-840a-9a67-9074 1
inserting weapon The Bloody Twins 2+ 4 -1 2 5ed6-602b-128c-54c5 6 828d-840a-9a67-9074
inserting keyword Assault
inserting keyword relationship 5ed6-602b-128c-54c5 70
inserting keyword Lethal Hits
inserting keyword relationship 5ed6-602b-128c-54c5 71
inserting keyword Pistol
inserting keyword relationship 5ed6-602b-128c-54c5 72
inserting weapon The Sword of Asur 2+ 6 -3 3 25e2-1809-549b-55e 6 828d-840a-9a67-9074
inserting keyword Devastating Wounds
inserting keyword relationship 25e2-1809-549b-55e 73
inserting unit cost 828d-840a-9a67-9074 0 default
inserting unit Autarch Skyrunner  69f4-c990-93b4-20fc dfcf-1214-b57-2205
inserting model 1d04-ad57-ec7c-8c47 Autarch Skyrunner 14" 4 5 6+ 3+ 69f4-c990-93b4-20fc 2
inserting weapon Twin Shuriken Catapult 2+ 4 -1 1 20e3-117a-27ce-4adb 2 69f4-c990-93b4-20f